<a href="https://colab.research.google.com/github/narender5162/Project-Data-science-imp/blob/main/Project_Data_Science_impo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
#importing library
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from plotly import express as px


In [52]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Importing file

In [53]:
file_path = '/content/drive/MyDrive/international-visitors-london-raw.csv'

In [54]:
# Load dataset with encoding fix (handling special characters)
df = pd.read_csv(file_path, encoding = "ISO-8859-1")


In [55]:
# Display the first few rows
print (df.head())

   year        quarter   market     dur_stay mode        purpose     area  \
0  2002  January-March  Belgium  1-3  nights  Air        Holiday   LONDON   
1  2002  January-March  Belgium  1-3  nights  Air       Business   LONDON   
2  2002  January-March  Belgium  1-3  nights  Air            VFR   LONDON   
3  2002  January-March  Belgium  1-3  nights  Air  Miscellaneous   LONDON   
4  2002  January-March  Belgium  1-3  nights  Sea       Business   LONDON   

   Visits (000s)  Spend (£m)  Nights (000s)  sample  
0       3.572186    0.969138       6.954456       5  
1       9.284226    2.399577      12.604959      19  
2       0.877182    0.089833       2.153128       3  
3       0.163874    0.010160       0.163874       1  
4       1.648670    0.016789       1.650300       1  


In [56]:
# Display the last few rows
print(df.tail())

        year        quarter        market     dur_stay mode        purpose  \
61457  2020P  January-March  Other Africa  4-7  nights  Air  Miscellaneous   
61458  2020P  January-March  Other Africa  8-14 nights  Air        Holiday   
61459  2020P  January-March  Other Africa  8-14 nights  Air            VFR   
61460  2020P  January-March  Other Africa  15+  nights  Air        Holiday   
61461  2020P  January-March  Other Africa  15+  nights  Air            VFR   

          area  Visits (000s)  Spend (£m)  Nights (000s)  sample  
61457   LONDON       1.695331    1.103167       8.831038       2  
61458   LONDON       1.486972    2.022254       7.836555       1  
61459   LONDON       2.416554    1.847152      63.894432       2  
61460   LONDON       2.472653    1.006109      35.080377       2  
61461   LONDON       4.124325    3.960705     134.909755       4  


In [57]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61462 entries, 0 to 61461
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   year           61462 non-null  object 
 1   quarter        61462 non-null  object 
 2   market         61462 non-null  object 
 3   dur_stay       61462 non-null  object 
 4   mode           61462 non-null  object 
 5   purpose        61462 non-null  object 
 6   area           61462 non-null  object 
 7   Visits (000s)  61462 non-null  float64
 8   Spend (£m)     61462 non-null  float64
 9   Nights (000s)  61462 non-null  float64
 10  sample         61462 non-null  int64  
dtypes: float64(3), int64(1), object(7)
memory usage: 5.2+ MB
None


In [58]:
# Data Cleaning
# Standardize column names (remove spaces and special characters correctly)
df.columns = (df.columns.str.strip()
              .str.lower()
              .str.replace(" ", "_")
              .str.replace(r"[()]", "", regex=True))  # Fix for FutureWarning

# Convert 'year' to integer type (handle non-numeric values properly)
df['year'] = pd.to_numeric(df['year'], errors='coerce')

# Check for missing values
missing_values = df.isnull().sum()

# Check for duplicate rows
duplicate_rows = df.duplicated().sum()

# Display results
missing_values, duplicate_rows


(year           630
 quarter          0
 market           0
 dur_stay         0
 mode             0
 purpose          0
 area             0
 visits_000s      0
 spend_£m         0
 nights_000s      0
 sample           0
 dtype: int64,
 0)

In [59]:
# Check rows with missing 'year' values
missing_year_rows = df[df['year'].isnull()]



In [60]:
# Display sample rows with missing 'year'
missing_year_rows.head()


,year,quarter,market,dur_stay,mode,purpose,area,visits_000s,spend_£m,nights_000s,sample
60832,NaN,January-March,Belgium,1-3 nights,Air,Holiday,LONDON,3.465766,1.224073,6.045668,3
60833,NaN,January-March,Belgium,1-3 nights,Air,Business,LONDON,2.141452,0.294101,2.243731,3
60834,NaN,January-March,Belgium,1-3 nights,Air,VFR,LONDON,1.531162,0.482316,4.593487,1
60835,NaN,January-March,Belgium,1-3 nights,Air,Miscellaneous,LONDON,0.818672,0.078494,0.818672,2
60836,NaN,January-March,Belgium,1-3 nights,Sea,Holiday,LONDON,1.571382,0.047141,3.142765,1


In [61]:
# Drop rows where 'year' is missing
df_cleaned = df.dropna(subset=['year'])

# Confirm that missing values are handled
missing_values_after = df_cleaned.isnull().sum()

# Display updated dataset info
df_cleaned_info = df_cleaned.info()

missing_values_after, df_cleaned_info


<class 'pandas.core.frame.DataFrame'>
Index: 60832 entries, 0 to 60831
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   year         60832 non-null  float64
 1   quarter      60832 non-null  object 
 2   market       60832 non-null  object 
 3   dur_stay     60832 non-null  object 
 4   mode         60832 non-null  object 
 5   purpose      60832 non-null  object 
 6   area         60832 non-null  object 
 7   visits_000s  60832 non-null  float64
 8   spend_£m     60832 non-null  float64
 9   nights_000s  60832 non-null  float64
 10  sample       60832 non-null  int64  
dtypes: float64(4), int64(1), object(6)
memory usage: 5.6+ MB


(year           0
 quarter        0
 market         0
 dur_stay       0
 mode           0
 purpose        0
 area           0
 visits_000s    0
 spend_£m       0
 nights_000s    0
 sample         0
 dtype: int64,
 None)

In [62]:
# Convert 'year' to integer type
df_cleaned = df.dropna(subset=['year']).copy()  # Best practice
df_cleaned['year'] = df_cleaned['year'].astype(int)  # Safe modification



In [63]:
# Check unique values in categorical columns for inconsistencies
categorical_columns = ['quarter', 'market', 'dur_stay', 'mode', 'purpose', 'area']
unique_values = {col: df_cleaned[col].unique() for col in categorical_columns}

unique_values


{'quarter': array(['January-March', 'April-June', 'July-September',
        'October-December'], dtype=object),
 'market': array(['Belgium', 'Luxembourg', 'France', 'Germany', 'Italy',
        'Netherlands', 'Denmark', 'Greece', 'Spain', 'Portugal', 'Austria',
        'Sweden', 'Finland', 'Irish Republic', 'Turkey', 'Switzerland',
        'Norway', 'Iceland', 'Czech Republic', 'Poland', 'Hungary',
        'Russia', 'Other Western Europe', 'Other Eastern Europe', 'USA',
        'Canada', 'Mexico', 'Brazil', 'Argentina',
        'Other Central & South America', 'Hong Kong', 'Malaysia',
        'Thailand', 'Singapore', 'Taiwan', 'China', 'Other Asia', 'Japan',
        'South Korea', 'Australia', 'New Zealand', 'United Arab Emirates',
        'Saudi Arabia', 'Kuwait', 'Other Middle East', 'Egypt', 'Israel',
        'India', 'Pakistan', 'South Africa', 'Other Southern Africa',
        'Nigeria', 'Kenya', 'Other Africa', 'Bulgaria', 'Romania',
        'Serbia', 'Chile', 'Indonesia', 'Bahrain

In [64]:
# Fix 'area' column by stripping spaces
df_cleaned['area'] = df_cleaned['area'].str.strip()

# Confirm the fix
df_cleaned['area'].unique()


array(['LONDON'], dtype=object)